In [132]:
import gurobipy as gp
import itertools as it

"""
implementation of solving profit maximization problem
author(s): 
yk796@cornell.edu
"""

'\nimplementation of solving profit maximization problem\nauthor(s): \nyk796@cornell.edu\n'

In [133]:
# input
n_nodes = 4
n_alternative = 3
O_demand = [1, 2, 0, 0]
D_demand = [0, 3, 0, 0]
assert sum(O_demand) == sum(D_demand)
bpr_a = 2
bpr_b = 3

nodes = list(range(1, n_nodes+1))
alternatives = list(range(1, n_alternative+1))
arcs = list(it.permutations(nodes, 2))
#ods = list(it.permutations(nodes, 2))
ods = [(id1+1, id2+1) for id1, o in enumerate(O_demand) for id2, d in enumerate(D_demand) if o>0 or d>0 if id1 != id2]
b = {key:2 for key in list(it.product(ods, alternatives))}
ods

[(1, 2), (1, 3), (1, 4), (2, 1), (2, 3), (2, 4), (3, 2), (4, 2)]

In [134]:
def profit_maximization(n_nodes, n_alternative, O_demand, D_demand, b, bpr_a, bpr_b):
    m = gp.Model()
    m.Params.DualReductions = 0 # to determine if the model is infeasible or unbounded
    #m.Params.OutputFlag = 0

    m._O = list(range(1, len(O_demand)+1))
    m._D = list(range(1, len(D_demand)+1))
    m._b = b
    m._nodes = list(range(1, n_nodes+1))
    m._alternatives = list(range(1, n_alternative+1))
    m._arcs = list(it.permutations(m._nodes, 2))
    m._ods = [(id1+1, id2+1) for id1, o in enumerate(O_demand) for id2, d in enumerate(D_demand) if o>0 or d>0 if id1 != id2]

    m._gvars = m.addVars(list(it.product(m._ods, m._alternatives)), vtype=gp.GRB.CONTINUOUS, lb=0, name='g')
    m._fvars = m.addVars(list(it.product(m._arcs, m._ods, m._alternatives)), vtype=gp.GRB.CONTINUOUS, lb=0, name='f')
    m._vvars = m.addVars(m._arcs, vtype=gp.GRB.CONTINUOUS, lb=0, name='v')

    # create auxiliary variables to deal with non-linear objective function
    m._lngvars = m.addVars(list(it.product(m._ods, m._alternatives)), vtype=gp.GRB.CONTINUOUS, name='lng')
    m._profit_extracting = m.addVar(vtype=gp.GRB.CONTINUOUS, lb = 0, name='extracting')
    m._profit_extracting_log = m.addVar(vtype=gp.GRB.CONTINUOUS, name='extracting_log')
    m._bpr = m.addVars(m._arcs, vtype=gp.GRB.CONTINUOUS, name='extracting')

    """add constraints"""
    # relationship between g and f
    for j in m._alternatives:
        for (s, t) in m._ods:
            lhs = gp.quicksum(m._fvars[(u, v), (s, t), j] for (u, v) in m._arcs if u == s)
            rhs = m._gvars[(s, t), j]
            m.addConstr(lhs == rhs)

    # satisfy origin demand
    for idx, o in enumerate(m._O):
        lhs = gp.quicksum(m._fvars[(u, v), (s, t), j] for (u, v) in m._arcs for (s, t) in m._ods for j in m._alternatives if u == o if s == o)
        rhs = O_demand[idx]
        m.addConstr(lhs == rhs)

    # satisfy destin demand
    for idx, d in enumerate(m._D):
        lhs = gp.quicksum(m._fvars[(u, v), (s, t), j] for (u, v) in m._arcs for (s, t) in m._ods for j in m._alternatives if v == d if t != d)
        rhs = D_demand[idx]
        m.addConstr(lhs == rhs)

    # flow conservation
    for j in m._alternatives:
        for (s, t) in m._ods:
            for v in m._nodes:
                if (v != s) and (v != t):
                    lhs = gp.quicksum(m._fvars[(u, v) , (s, t), j] for u in m._nodes if u != v)
                    rhs = gp.quicksum(m._fvars[(v, u) , (s, t), j] for u in m._nodes if u != v)
                    m.addConstr(lhs == rhs)

    # relationship to v_a
    for a in m._arcs:
        lhs = m._vvars[a]
        rhs = gp.quicksum(m._fvars[a , (s, t), j] for (s, t) in m._ods for j in m._alternatives)
        m.addConstr(lhs == rhs)

    # objective function

    for j in m._alternatives:
        for (s, t) in m._ods:
            m.addGenConstrLog(m._gvars[(s, t), j], m._lngvars[(s, t), j]) # TODO: set 0 if gvar = 0 

    m.addConstr(m._profit_extracting == 1 - gp.quicksum(m._gvars[(s, t), j] for (s, t) in m._ods for j in m._alternatives)/sum(m._O))
    m.addGenConstrLog(m._profit_extracting, m._profit_extracting_log)

    # Choose more general bpr function
    for a in m._arcs:
        lhs = m._bpr[a]
        rhs = 0.5 * bpr_a * m._vvars[a] * m._vvars[a] + bpr_b * m._vvars[a]
        m.addConstr(lhs == rhs)
        

    m.setObjective(- gp.quicksum(m._gvars[(s, t), j]*m._b[(s, t), j] for (s, t) in m._ods for j in m._alternatives)
                   + gp.quicksum(m._gvars[(s, t), j]*m._lngvars[(s, t), j] for (s, t) in m._ods for j in m._alternatives)
                   - m._profit_extracting_log
                   + gp.quicksum(m._bpr[a] for a in m._arcs)
                   )


    m.update()
    m.optimize()

    m.computeIIS() # this helps us to identify constraints that are responsible to make the model infeasible.
    m.write("model.ilp")

    # if m.Status == 3:
    #     return None
    # else:
    #     return m._fvars, m.ObjVal

profit_maximization(n_nodes, n_alternative, O_demand, D_demand, b, bpr_a, bpr_b)

Set parameter DualReductions to value 0
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) Gold 6244 CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 16 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 93 rows, 362 columns and 997 nonzeros
Model fingerprint: 0x17a8ac04
Model has 24 quadratic objective terms
Model has 12 quadratic constraints
Model has 25 general constraints
Variable types: 362 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 57 rows and 243 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 32 available processors)

Solution count 0

Model is inf